In [1]:
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.models import Sequential
from qkeras import QActivation, quantized_relu, QDense, quantized_bits, QDenseBatchnorm
from tensorflow.python.keras.utils.np_utils import to_categorical
import numpy as np
import setGPU
physical_devices = tf.config.list_physical_devices('GPU') 
if physical_devices: 
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

def get_model_ut(input_shape):
    model = Sequential() 
    model.add(QDenseBatchnorm(16, input_shape=input_shape, kernel_quantizer=quantized_bits(16, 6, alpha=1),
                               bias_quantizer=quantized_bits(16, 6, alpha=1), name="qc2dbn0"))
    model.add(QActivation(quantized_relu(6), name="relu0"))
    
    model.add(QDenseBatchnorm(16, kernel_quantizer=quantized_bits(16, 6, alpha=1),
                               bias_quantizer=quantized_bits(16, 6, alpha=1), name="qc2dbn1"))
    model.add(QActivation(quantized_relu(6), name="relu1"))
    
    model.add(QDenseBatchnorm(24, kernel_quantizer=quantized_bits(16, 6, alpha=1),
                               bias_quantizer=quantized_bits(16, 6, alpha=1), name="qc2dbn2"))
    model.add(QActivation(quantized_relu(6), name="relu2"))
    
    model.add(Flatten())
    
    model.add(QDense(42, kernel_quantizer=quantized_bits(6, 0, alpha=1), bias_quantizer=quantized_bits(6, 0, alpha=1),
                     kernel_initializer='lecun_uniform', name="dense0"))

    model.add(QActivation(quantized_relu(6), name="relu3"))
    
    model.add(QDense(64, kernel_quantizer=quantized_bits(6, 0, alpha=1), bias_quantizer=quantized_bits(6, 0, alpha=1),
                     kernel_initializer='lecun_uniform', name="dense1"))

    model.add(QActivation(quantized_relu(6), name="relu4"))
    
    model.add(QDense(5, kernel_quantizer=quantized_bits(6, 0, alpha=1), bias_quantizer=quantized_bits(6, 0, alpha=1),
                     kernel_initializer='lecun_uniform'))
    
    model.add(Activation(activation='softmax', name='softmax'))
        
    return model


def get_model_ref(input_shape):
    model = Sequential()
    model.add(QDense(16, input_shape=input_shape, kernel_quantizer=quantized_bits(16, 6, alpha=1),
                      bias_quantizer=quantized_bits(16, 6, alpha=1), name="qc2d0"))
    model.add(BatchNormalization(name="bn0"))
    model.add(QActivation(quantized_relu(6), name="relu0"))
    
    model.add(QDense(16, kernel_quantizer=quantized_bits(16, 6, alpha=1),
                      bias_quantizer=quantized_bits(16, 6, alpha=1), name="qc2d1"))
    model.add(BatchNormalization(name="bn1"))
    model.add(QActivation(quantized_relu(6), name="relu1"))
    
    model.add(QDense(24, kernel_quantizer=quantized_bits(16, 6, alpha=1),
                      bias_quantizer=quantized_bits(16, 6, alpha=1), name="qc2d2"))
    model.add(BatchNormalization(name="bn2"))
    model.add(QActivation(quantized_relu(6), name="relu2"))
    
    model.add(Flatten())
    
    model.add(QDense(42, kernel_quantizer=quantized_bits(6, 0, alpha=1), bias_quantizer=quantized_bits(6, 0, alpha=1),
                     kernel_initializer='lecun_uniform', name="dense0"))
    # model.add(BatchNormalization(name="bn3"))
    model.add(QActivation(quantized_relu(6), name="relu3"))
    
    model.add(QDense(64, kernel_quantizer=quantized_bits(6, 0, alpha=1), bias_quantizer=quantized_bits(6, 0, alpha=1),
                     kernel_initializer='lecun_uniform', name="dense1"))
    # model.add(BatchNormalization(name="bn4"))
    model.add(QActivation(quantized_relu(6), name="relu4"))
    
    model.add(QDense(5, kernel_quantizer=quantized_bits(6, 0, alpha=1), bias_quantizer=quantized_bits(6, 0, alpha=1),
                     kernel_initializer='lecun_uniform'))
    model.add(Activation(activation='softmax', name='softmax'))
        
    return model

def dummy_model_ut(input_shape):
    model = Sequential()
    model.add(
        QDenseBatchnorm(16, input_shape=input_shape,
                         kernel_initializer=tf.keras.initializers.RandomUniform(minval=-1, maxval=1, seed=96),
                         bias_initializer=tf.keras.initializers.RandomUniform(minval=-1, maxval=1, seed=96),
                         beta_initializer=tf.keras.initializers.RandomUniform(minval=-4, maxval=4, seed=96),
                         gamma_initializer=tf.keras.initializers.RandomUniform(minval=-4, maxval=4, seed=96),
                         moving_mean_initializer=tf.keras.initializers.RandomUniform(minval=-4, maxval=4, seed=96),
                         moving_variance_initializer=tf.keras.initializers.RandomUniform(minval=0, maxval=4, seed=96),
                         kernel_quantizer=quantized_bits(16, 6, alpha=1),
                         bias_quantizer=quantized_bits(16, 6, alpha=1),
                         name="qdensebatchnorm")
        )
    
    model.build(input_shape=input_shape)
    
    return model

def dummy_model_ref(input_shape):
    model = Sequential()
    model.add(
        QDense(16, input_shape=input_shape,
                 kernel_initializer=tf.keras.initializers.RandomUniform(minval=-1, maxval=1, seed=96),
                 bias_initializer=tf.keras.initializers.RandomUniform(minval=-1, maxval=1, seed=96),
                 kernel_quantizer=quantized_bits(16, 6, alpha=1),
                 bias_quantizer=quantized_bits(16, 6, alpha=1),
                 name="qdense")
        )
    model.add(
        BatchNormalization(
                 beta_initializer=tf.keras.initializers.RandomUniform(minval=-4, maxval=4, seed=96),
                 gamma_initializer=tf.keras.initializers.RandomUniform(minval=-4, maxval=4, seed=96),
                 moving_mean_initializer=tf.keras.initializers.RandomUniform(minval=-4, maxval=4, seed=96),
                 moving_variance_initializer=tf.keras.initializers.RandomUniform(minval=0, maxval=4, seed=96),
                 name='bn')
    )
    
    model.build(input_shape=input_shape)
    
    return model

setGPU: Setting GPU to: 0


2022-01-09 07:00:12.567341: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-09 07:00:12.577561: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-09 07:00:12.578256: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


In [2]:
from sklearn.datasets import fetch_openml
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
import numpy as np
input_shape = (16,)

data = fetch_openml('hls4ml_lhc_jets_hlf')
X, y = data['data'], data['target']
le = LabelEncoder()
y = le.fit_transform(y)
y = to_categorical(y, 5)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [3]:
dummy_ref = dummy_model_ref(input_shape)
dummy_ut  = dummy_model_ut(input_shape)

y_ref = dummy_ref.predict(X_test[:1000])
y_ut  = dummy_ut.predict(X_test[:1000])

max_abs_error = (np.abs(y_ref - y_ut)).max()
mean_rel_error = (np.abs(y_ut[y_ut != 0] - y_ref[y_ut != 0]) / np.abs(y_ut[y_ut != 0])).mean() * 100

print('Max absolute error:  {}'.format(max_abs_error))
print('Mean relative error: {} %'.format(mean_rel_error))

2022-01-09 07:00:33.532261: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-01-09 07:00:33.533661: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-09 07:00:33.534450: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-09 07:00:33.535255: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zer

Max absolute error:  1041.022216796875
Mean relative error: 1314.5424842834473 %


In [4]:
from sklearn.metrics import accuracy_score
from tensorflow.keras.optimizers import Adam
from qkeras.utils import load_qmodel
import os

def train_or_load(name, model):
    if not os.path.isdir(name):
        adam = Adam(lr=0.0001)
        model.compile(optimizer=adam, loss=['categorical_crossentropy'], metrics=['accuracy'])
        model.fit(X_train, y_train, batch_size=128,
                  epochs=30, validation_split=0.25, shuffle=True,)
    else:
        model = tf.keras.models.load_model(name + "/KERAS_check_best_model.h5",custom_objects={'quantized_relu':quantized_relu,'QDense': QDense, 'QActivation': QActivation, 'QDenseBatchnorm': QDenseBatchnorm})
    return model

ut_model_dir = "model_ut_real"
ut_model  = train_or_load('keras_' + ut_model_dir, get_model_ut(input_shape))

y_keras_ut = ut_model.predict(X_test)

acc_keras_ut = accuracy_score(np.argmax(y_test, axis=1), np.argmax(y_keras_ut, axis=1))

/opt/conda/envs/tiny_qdense_batchnorm/lib/python3.7/site-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Epoch 1/30
3891/3891 [==============================] - 30s 7ms/step - loss: 0.9855 - accuracy: 0.6524 - val_loss: 0.8486 - val_accuracy: 0.7051
Epoch 2/30
3891/3891 [==============================] - 28s 7ms/step - loss: 0.8775 - accuracy: 0.6889 - val_loss: 0.8333 - val_accuracy: 0.7066
Epoch 3/30
3891/3891 [==============================] - 28s 7ms/step - loss: 0.8588 - accuracy: 0.6957 - val_loss: 0.8270 - val_accuracy: 0.7078
Epoch 4/30
3891/3891 [==============================] - 28s 7ms/step - loss: 0.8503 - accuracy: 0.6983 - val_loss: 0.8259 - val_accuracy: 0.7076
Epoch 5/30
3891/3891 [==============================] - 27s 7ms/step - loss: 0.8433 - accuracy: 0.7001 - val_loss: 0.8206 - val_accuracy: 0.7093
Epoch 6/30
3891/3891 [==============================] - 27s 7ms/step - loss: 0.8390 - accuracy: 0.7010 - val_loss: 0.8183 - val_accuracy: 0.7091
Epoch 7/30
3891/3891 [==============================] - 27s 7ms/step - loss: 0.8337 - accuracy: 0.7022 - val_loss: 0.8203 - val_ac

In [5]:
ref_model_dir = "model_ref_real"
ref_model  = train_or_load('keras_' + ref_model_dir, get_model_ref(input_shape))

y_keras_ref = ref_model.predict(X_test)

acc_keras_ref = accuracy_score(np.argmax(y_test, axis=1), np.argmax(y_keras_ref, axis=1))

/opt/conda/envs/tiny_qdense_batchnorm/lib/python3.7/site-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Epoch 1/30
3891/3891 [==============================] - 27s 6ms/step - loss: 0.9637 - accuracy: 0.6681 - val_loss: 0.8499 - val_accuracy: 0.7049
Epoch 2/30
3891/3891 [==============================] - 24s 6ms/step - loss: 0.8713 - accuracy: 0.6930 - val_loss: 0.8384 - val_accuracy: 0.7055
Epoch 3/30
3891/3891 [==============================] - 24s 6ms/step - loss: 0.8584 - accuracy: 0.6967 - val_loss: 0.8325 - val_accuracy: 0.7059
Epoch 4/30
3891/3891 [==============================] - 24s 6ms/step - loss: 0.8533 - accuracy: 0.6985 - val_loss: 0.8310 - val_accuracy: 0.7063
Epoch 5/30
3891/3891 [==============================] - 24s 6ms/step - loss: 0.8500 - accuracy: 0.6991 - val_loss: 0.8272 - val_accuracy: 0.7073
Epoch 6/30
3891/3891 [==============================] - 24s 6ms/step - loss: 0.8464 - accuracy: 0.7005 - val_loss: 0.8264 - val_accuracy: 0.7078
Epoch 7/30
3891/3891 [==============================] - 24s 6ms/step - loss: 0.8437 - accuracy: 0.7010 - val_loss: 0.8242 - val_ac

In [6]:
print('Keras  model QDenseBatchnorm:    {:,.2f}'.format(acc_keras_ut))
print('Keras  model QDense + Batchnorm: {:,.2f}'.format(acc_keras_ref))

Keras  model QDenseBatchnorm:    0.72
Keras  model QDense + Batchnorm: 0.71
